In [3]:
# Useful starting lines
%matplotlib inline
import numpy as np
import itertools
import matplotlib.pyplot as plt
import csv
from helpers import *

ModuleNotFoundError: No module named 'helpers'

In [2]:
def load_data_train(path_dataset,sub_sample=True, add_outlier=False):
    """Load data."""
    data = np.genfromtxt(
        path_dataset, delimiter=",", dtype=str,  skip_header=1)
    ids = data[:,0]
    labels = data[:,1]
    labels[labels=='s']=1
    labels[labels=='b']=-1
    #labels[labels=='?']=0
    labels = np.asarray(labels, dtype=float)
    data = np.delete(data, [0,1], 1)
    data = np.asarray(data, dtype=float)
    return data, labels, ids

In [3]:
def load_data_test(path_dataset,sub_sample=True, add_outlier=False):
    """Load data and convert it to the metric system."""
    data = np.genfromtxt(
        path_dataset, delimiter=",", dtype=str,  skip_header=1)
    ids = data[:,0]
    labels = data[:,1]
    data = np.delete(data, [0,1], 1)
    data = np.asarray(data, dtype=float)
    return data, labels, ids

Prediction of :

- type s is assigned value 0
- type b is assigned value 1

In [8]:
# Load data
train_data, train_labels, ids = load_data_train('train.csv')

In [5]:
def clean_data(data):
    # Remove columns with more than 50% of -999
    dirty_cols = np.where(np.sum(train_data == -999, axis=0)/train_data.shape[0] < 0.5, True, False)
    data = data[:, dirty_cols]
    # Replace -999 by nan
    data = np.where(data == -999, np.nan, data)
    # Compute the columns means without nan values 
    means = np.nanmean(data, axis=0)
    #Find indices that you need to replace
    inds = np.where(np.isnan(data))
    #Place column means in the indices. Align the arrays using take
    data[inds] = np.take(means, inds[1])
    return data

In [6]:
def clean_data_old(data):
    # remove the columns containing a -999 value
    valid_cols = np.all(data!=-999, axis=0)
    return data[:,valid_cols]

In [7]:
# Standardize the data
def standardize(x):
    #Standardize the original data set.
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x

In [8]:
new_data = standardize(clean_data(train_data))

**Logistic Regression**

In [9]:
def calculate_loss_reg(labels, data, w, lambda_):
    """compute the cost by negative log likelihood."""
    loss = np.sum(np.logaddexp(0, data @ w) + labels * data.dot(w)) + lambda_*np.linalg.norm(w)**2
    return loss

In [10]:
def sigmoid(t):
    return 1.0 / (1 + np.exp(-t))

In [11]:
def calculate_loss_old(labels, data, w):
    """compute the cost by negative log likelihood."""
    loss = -np.sum(labels * np.log(sigmoid(data @ w)) + (1 - labels) * np.log(1 - sigmoid(data @ w)))
    return loss

In [12]:
def calculate_loss(labels, data, w):
    """compute the cost by negative log likelihood."""
    loss = np.sum(np.logaddexp(0, data @ w) + labels * data.dot(w))
    return loss

In [13]:
def initialize_weights(tx):
    initial_w = np.random.normal(0., 0.1, [tx.shape[1],])
    return initial_w

In [14]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    """ Training function for binary class logistic regression. 
    
    Args:
        y (np.array): Labels of shape (N, ).
        tx (np.array): Dataset of shape (N, D).
        initial_w (np.array): Initial weights of shape (D,)
        max_iters (integer): Maximum number of iterations.
        gamma (integer): Step size
    Returns:
        np.array: weights of shape(D, )
    """  
    def sigmoid(t):
        """apply sigmoid function on t."""
        return 1.0 / (1 + np.exp(-t))

    threshold = 1e-8
    losses = []
    
    w = initialize_weights(tx)
    
    for it in range(max_iters):
        #loss = np.sum(np.logaddexp(0, tx.dot(w)) + y * tx.dot(w))
        grad = tx.T.dot(sigmoid(tx.dot(w)) - y)
        w -= gamma * grad
        # log info
        if it % 100 == 0:
            print(f"Current iteration={it}")
        #    print("Current iteration={i}, loss={l}".format(i=it, l=loss))
        # converge criterion
        #losses.append(loss)
        #if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            #break
    loss = calculate_loss_reg(y, tx, w)
    print("loss={l}".format(l=loss))
    return (w, loss)

In [15]:
def reg_logistic_regression(y, tx, lambda_, max_iters, gamma):
    """ Training function for binary class logistic regression. 
    
    Args:
        y (np.array): Labels of shape (N, ).
        tx (np.array): Dataset of shape (N, D).
        lambda_ (integer): Regularization factor
        initial_w (np.array): Initial weights of shape (D,)
        max_iters (integer): Maximum number of iterations.
        gamma (integer): Step size
    Returns:
        np.array: weights of shape(D, )
    """  
    def sigmoid(t):
        """apply sigmoid function on t."""
        return 1.0 / (1 + np.exp(-t))

    threshold = 1e-8
    losses = []
    
    w = initialize_weights(tx)
    
    for it in range(max_iters):
        #loss = -np.sum(y * np.log(sigmoid(tx @ w)) + (1 - y) * np.log(1 - sigmoid(tx @ w))) + lambda_*np.linalg.norm(w)**2
        grad = tx.T.dot(sigmoid(tx.dot(w)) - y) + 2*lambda_*w
        w -= gamma * grad
        # log info
        #if it % 100 == 0:
            #print("Current iteration={i}, loss={l}".format(i=it, l=loss))
            #print(f"Current iteration={it}")
        # converge criterion
        #losses.append(loss)
        #if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
        #    break
    loss = calculate_loss_reg(y, tx, w, lambda_)
    print("loss={l}".format(l=loss))
    return (w, loss)

In [16]:
def ridge_regression_ls(x, y, lambda_):
    """ridge regression for least squares loss function"""
    A = x.T.dot(x) + 2*len(x)*lambda_*np.identity(x.shape[1])
    B = x.T.dot(y)
    return np.linalg.solve(A,B)

In [17]:
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [18]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [19]:
def cross_validation_old(x, y, ratio, lambda_, max_iters, gamma):
    num_row = len(y)
    index_split = int(np.floor(ratio*num_row))
    k = len(y)-index_split-1
    ws = []
    losses = []
    for i in range(0,k):
        index_te = indices[i:i+index_split]
        x_te = x[index_te]
        y_te = y[index_te]
        x_tr = np.delete(x, index_te, 0)
        y_tr = np.delete(y, index_te, 0)
        w, loss_tr = reg_logistic_regression(y_tr, x_tr, lambda_, max_iters, gamma)
        loss_te = calculate_loss(y_te, x_te)
        ws.append(w)
        losses.append(loss)
    return ws, losses

In [20]:
def cross_validation(y, x, k_indices, k, lambda_, max_iters, gamma):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train
    te_index = k_indices[k]
    tr_index = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_index = tr_index.reshape(-1)
    y_te = y[te_index]
    y_tr = y[tr_index]
    x_te = x[te_index]
    x_tr = x[tr_index]
    # weights and training loss for logistic regression model:
    w, loss_tr = reg_logistic_regression(y_tr, x_tr, lambda_, max_iters, gamma) 
    # calculate the loss for test data:
    loss_te = calculate_loss_reg(y_te, x_te, w, lambda_)
    return loss_tr, loss_te, w

In [21]:
def best_lambda_selection(y, x, k_fold, max_iters, gamma):
    seed = 12
    lambdas = np.logspace(-4, 0, 3) #change last parameter to 10 if it works
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    loss_training = []
    loss_test = []
    # cross validation
    for lambda_ in lambdas:
        print(f"Current lambda={lambda_}")
        loss_tr_tmp = []
        loss_te_tmp = []
        for k in range(k_fold):
            print(f"Current iteration={k}")
            loss_tr, loss_te,_ = cross_validation(y, x, k_indices, k, lambda_, max_iters, gamma)
            loss_tr_tmp.append(loss_tr)
            loss_te_tmp.append(loss_te)
        loss_training.append(np.mean(loss_tr_tmp))
        loss_test.append(np.mean(loss_te_tmp))
        
    ind_best_lambda = np.argmin(loss_test)
    best_lambda = lambdas[ind_best_lambda]
    
    return best_lambda

In [22]:
def predict_logistic(tx, w):
    def sigmoid(t):
        return 1.0 / (1 + np.exp(-t))
    y = sigmoid(tx @ w)
    # s = 1 , b = -1
    y[y < 0.5] = 1
    y[y >= 0.5] = -1
    return y

In [23]:
def accuracy(a, b):
    return np.sum(a == b)/a.shape[0]

In [24]:
def csv_submission(ids, y_pred, name):

    with open (name, 'w') as csvfile:
        fd = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fd)
        writer.writeheader()
        for r1, r2 in zip (ids, y_pred):
            writer.writerow({'Id' : int(r1), 'Prediction': str(r2)})

**Testing on test data:**

In [25]:
# Add bias to data
tx = np.c_[np.ones((train_labels.shape[0], 1)), new_data]

In [26]:
#find optimal value for lambda (such that loss is minimized):
lambda_ = best_lambda_selection(train_labels, tx, 4, 5000, 0.1)
print("Optimal lambda: ",lambda_)
#Compute model weights and loss with optimal lambda:
trained_weights, train_loss = reg_logistic_regression(train_labels, tx, lambda_, max_iters=5000, gamma=0.1)

Current lambda=0.0001
Current iteration=0


<ipython-input-15-0cad211ba73a>:16: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))


loss=5340012927409.967
Current iteration=1
loss=5330004467709.197
Current iteration=2
loss=5384708190546.278
Current iteration=3
loss=5375161714547.664
Current lambda=0.01
Current iteration=0
loss=779417096984.3026
Current iteration=1
loss=778050813501.0726
Current iteration=2
loss=785713013933.1128
Current iteration=3
loss=784524777305.6493
Current lambda=1.0
Current iteration=0
loss=7794320816.6141205
Current iteration=1
loss=7780995663.842256
Current iteration=2
loss=7857124945.090847
Current iteration=3
loss=7845630214.326981
Optimal lambda:  1.0
loss=13901029251.560066


In [27]:
#Load and standardize test data:
test_data, labels, ids = load_data('test.csv')
test_data = standardize(clean_data(test_data))

In [28]:
#Generate labels for test data:
tx_test = np.c_[np.ones((test_data.shape[0], 1)), test_data]
predicted_labels = predict_logistic(tx_test, trained_weights)

<ipython-input-22-ecbc95861a45>:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))


In [29]:
csv_submission(ids, predicted_labels, 'Cross_Validation_Prediction.csv')